In [89]:
import warnings
import pandas as pd
import numpy as np
from string import digits
warnings.simplefilter('ignore')

In [90]:
# Load csv files into dataframe
train = pd.read_csv("Resources/train.csv")
test = pd.read_csv("Resources/test.csv")

In [91]:
test_df=test.copy()
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [92]:
train['Cabin']=train.Cabin.replace('/','', regex=True)
train['Cabin']=train.Cabin.str.replace('\d+','')
# train.Cabin.unique()
test['Cabin']=test.Cabin.replace('/','', regex=True)
test['Cabin']=test.Cabin.str.replace('\d+','')

In [93]:
drop_cols = ["PassengerId", "Name"]
train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)

In [94]:
# Remove null values with 0, median value and 'False'
age_median = train.Age.median()
train.Age.fillna(age_median, inplace = True)
train.fillna(0, inplace = True)

test_age_median = test.Age.median()
test.Age.fillna(test_age_median, inplace = True)
test.fillna(0, inplace = True)

In [95]:
train.CryoSleep.replace({True:1,False:0}, inplace = True)
train.Transported.replace({True:1,False:0}, inplace = True)
# train.drop(columns=['Destination','VIP'], inplace = True)

test.CryoSleep.replace({True:1,False:0}, inplace = True)
# test.drop(columns=['Destination','VIP'], inplace = True)

In [96]:
test = pd.get_dummies(test)

train=pd.get_dummies(train)

In [97]:
X=train.drop(columns='Transported')
y=train.Transported

In [98]:
print(X.shape)
print(test.shape)

(8693, 34)
(4277, 34)


In [109]:
# Split X and y using train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.70, random_state=30)

# Get shape of train test data
print("train size X : ",X_train.shape)
print("train size y : ",y_train.shape)
print("test size X : ",X_test.shape)
print("test size y : ",y_test.shape)

train size X :  (6085, 34)
train size y :  (6085,)
test size X :  (2608, 34)
test size y :  (2608,)


In [110]:
from sklearn.preprocessing import StandardScaler
 
ss = StandardScaler()

# Fit and transform the data
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)

In [111]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train_rs = scaler.fit_transform(X_train)
X_test_rs = scaler.fit_transform(X_test)

In [112]:
from sklearn.ensemble import RandomForestClassifier

# Train and fit the model
rfc = RandomForestClassifier(n_estimators=125, 
                             random_state=1,min_samples_leaf=8)

rfc.fit(X_train, y_train)

importance = rfc.feature_importances_

print(f'Training score: {rfc.score(X_train, y_train)}')
print(f'Testing score: {rfc.score(X_test, y_test)}')
y_pred_rfc  =  rfc.predict(test)

spaceship_df = pd.DataFrame({'PassengerId':test_df.PassengerId, 'Transported':y_pred_rfc})
spaceship_df['Transported'] = spaceship_df.Transported.map({0:False, 1:True})
spaceship_df.to_csv('spaceship_rfc.csv', index = False)
y_pred_rfc

Training score: 0.8279375513557929
Testing score: 0.8059815950920245


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [113]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(iterations=130, learning_rate=0.1)
cbc.fit(X_train, y_train)

# Model Accuracy
print(f'Training Score: {cbc.score(X_train, y_train)}')
print(f'Testing Score: {cbc.score(X_test, y_test)}')
y_pred_cbc  =  cbc.predict(test)
print(y_pred_cbc)
spaceship_df = pd.DataFrame({'PassengerId':test_df.PassengerId, 'Transported':y_pred_cbc})
spaceship_df['Transported'] = spaceship_df.Transported.map({0:False, 1:True})
# save predicted data in csv file
spaceship_df.to_csv('spaceship_cbc.csv', index = False)

# predicted score 80.360

0:	learn: 0.6507275	total: 5.5ms	remaining: 710ms
1:	learn: 0.6143960	total: 11.8ms	remaining: 755ms
2:	learn: 0.5826983	total: 17.6ms	remaining: 745ms
3:	learn: 0.5603092	total: 24ms	remaining: 755ms
4:	learn: 0.5451975	total: 32ms	remaining: 801ms
5:	learn: 0.5302293	total: 38ms	remaining: 785ms
6:	learn: 0.5165656	total: 45.5ms	remaining: 799ms
7:	learn: 0.5068629	total: 50.6ms	remaining: 772ms
8:	learn: 0.4966525	total: 56.7ms	remaining: 763ms
9:	learn: 0.4878146	total: 61.7ms	remaining: 740ms
10:	learn: 0.4810808	total: 66.2ms	remaining: 717ms
11:	learn: 0.4740226	total: 71.2ms	remaining: 700ms
12:	learn: 0.4684418	total: 76.2ms	remaining: 686ms
13:	learn: 0.4639113	total: 81.2ms	remaining: 673ms
14:	learn: 0.4591584	total: 86.6ms	remaining: 664ms
15:	learn: 0.4551928	total: 91.4ms	remaining: 651ms
16:	learn: 0.4517686	total: 96.2ms	remaining: 640ms
17:	learn: 0.4486392	total: 101ms	remaining: 630ms
18:	learn: 0.4449632	total: 108ms	remaining: 632ms
19:	learn: 0.4432028	total: 113

In [114]:
from xgboost import XGBClassifier

xg_model = XGBClassifier(booster='gbtree',
                        learning_rate =0.08, n_estimators = 100, max_depth=6,
                        gamma=0.1, subsample = 0.7,colsample_bynode = 0.3,
                        nthread=4,reg_lambda=11,
                        scale_pos_weight = 1, seed = 27, num_parallel_tree=10
                        ).fit(X_train, y_train)

# from sklearn.metrics import mean_absolute_error
print(f'Training Score: {xg_model.score(X_train, y_train)}')
print(f'Testing Score: {xg_model.score(X_test, y_test)}')
y_pred_xgb=xg_model.predict(test)
spaceship_df = pd.DataFrame({'PassengerId':test_df.PassengerId, 'Transported':y_pred_xgb})
spaceship_df['Transported'] = spaceship_df.Transported.map({0:False, 1:True})
spaceship_df.to_csv('spaceship_xgb.csv', index = False)
y_pred_xgb

[18:27:12] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training Score: 0.8238290879211175
Testing Score: 0.8098159509202454


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)